## 光流
因为强度不变
$$
I(x,y,t)=I(x+d x,y+d y,t+d t)
$$
又因为泰勒展开
$$
I(x + dx, y + dy, t + dt) ≈ I(x, y, t) + dx * ∂I/∂x + dy * ∂I/∂y + dt * ∂I/∂t
$$
两边同时除以得到$dt$

$$
\begin{aligned}
&f_x u+f_y v+f_t& =0  \\
&f_x =\frac{\partial f}{\partial x};f_{y}=\frac{\partial f}{\partial y}  \\
& \\
&u={\frac{d x}{d t}};v={\frac{d y}{d t}}
\end{aligned}
$$

##  OpenCV中的Lucas-Kanade(密集光流)
    
    ```python
    cv.calcOpticalFlowPyrLK(prevImg, nextImg, prevPts[, nextPts[, status[, err[, winSize[, maxLevel[, criteria[, flags[, minEigThreshold]]]]]]]]) -> nextPts, status, err
    ```
    - prevImg: 前一帧图像
    - nextImg: 后一帧图像
    - prevPts: 前一帧图像中的点
    - nextPts: 后一帧图像中的点
    - status: 输出状态向量，如果找到了相应特征的流，则为1，否则为0
    - err: 输出误差向量
    - winSize: 搜索窗口的大小
    - maxLevel: 金字塔的最大层数
    - criteria: 终止条件
    - flags: 操作标志
    - minEigThreshold: 最小特征值阈值，小于此阈值的特征值将被忽略
    ```
    
    ```python



In [ ]:
import numpy as np
import cv2 as cv
import argparse
filename = '../data/car.avi'
cap = cv.VideoCapture(cv.samples.findFileOrKeep(filename))
#cap = cv.VideoCapture(0)

# 用于ShiTomasi拐点检测的参数:
# maxCorners - 保留的最大角点数。如果角点数超过了这个数目，就会按照角点强度进行排序。
# qualityLevel - 角点的最低质量，小于这个数的角点会被忽略。
# minDistance - 两个角点之间的最短欧式距离。
# blockSize - 计算导数自相关矩阵时指定的邻域范围。
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.63,
                       minDistance = 7,
                       blockSize = 7 )
# lucas kanade光流参数:
# winSize - 搜索窗口的大小，以像素为单位。
# maxLevel - 金字塔层数
# criteria - 停止迭代的准则:最大迭代次数10或者最大误差容限0.03
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))

# 创建一些随机的颜色
color = np.random.randint(0,255,(100,3))
# 拍摄第一帧并在其中找到拐角
ret, old_frame = cap.read()
old_gray = cv.cvtColor(old_frame, cv.COLOR_BGR2GRAY)
#goodFeaturesToTrack是ShiTomasi角点检测器的一个实现,参数是图像,角点数,质量,最小距离
# goodFeaturesToTrack中 **feature_params是一个字典,可以用**解包
p0 = cv.goodFeaturesToTrack(old_gray, mask = None, **feature_params)
# 创建用于作图的掩码图像 zeros_like是创建一个和old_frame一样大小的全0矩阵
mask = np.zeros_like(old_frame)
while(1):
    ret,frame = cap.read()
    frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    p1, st, err = cv.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

    good_new = p1[st==1]
    good_old = p0[st==1]
    # 绘制跟踪
    #enumerate(zip(good_new, good_old))是将good_new和good_old的元素组成一个元组,然后再将这些元组组成一个列表
    #zip是将两个列表组合成一个元组,enumerate是将一个列表组合成一个带索引的元组
    for i,(new,old) in enumerate(zip(good_new, good_old)):#为了遍历两个列表,用zip将两个列表组合成一个元组
        #ravel()是将多维数组转换为一维数组
        a,b = new.ravel()
        c,d = old.ravel()
        #转换为int
        a,b,c,d = int(a),int(b),int(c),int(d)
        #把老的点和新的点连起来
        mask = cv.line(mask, (a,b),(c,d), color[i].tolist(), 2)
        #在新的帧上画出新的点
        frame = cv.circle(frame,(a,b),5,color[i].tolist(),-1)
    #在新的帧上画出跟踪线
    img = cv.add(frame,mask)
    cv.imshow('frame',img)
    k = cv.waitKey(30) & 0xff
    if k == 27:
        break
    # 现在更新之前的帧和点
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1,1,2)
cap.release()
cv.destroyAllWindows()

##  OpenCV中的吗密集光流calcOpticalFlowFarneback()
```python
cv.calcOpticalFlowFarneback(prev, next, flow, pyr_scale, levels, winsize, iterations, poly_n, poly_sigma, flags[, flow]) -> flow
```
- prev: 前一帧图像
- next: 后一帧图像
- flow: 输出光流
- pyr_scale: 金字塔尺度因子
- levels: 金字塔层数
- winsize: 窗口大小
- iterations: 迭代次数
- poly_n: 像素领域大小，一般为5
- poly_sigma: 高斯标准差，一般为1.1
- flags: 操作标志
- flow: 输出光流
```

    


In [ ]:
import numpy as np
import cv2 as cv
cap = cv.VideoCapture(cv.samples.findFile("../data/vtest.avi"))
ret, frame1 = cap.read()
prvs = cv.cvtColor(frame1,cv.COLOR_BGR2GRAY)
hsv = np.zeros_like(frame1)
#这是什么语法?hsv[...,1]是什么意思?这是numpy的高级索引,意思是取hsv的第二个通道
hsv[...,1] = 255
while(1):
    ret, frame2 = cap.read()
    next = cv.cvtColor(frame2,cv.COLOR_BGR2GRAY)
    #计算光流
    flow = cv.calcOpticalFlowFarneback(prvs,next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    #计算角度和大小,cartToPolar是将笛卡尔坐标转换为极坐标.flow[...,0]是x方向的光流,flow[...,1]是y方向的光流
    mag, ang = cv.cartToPolar(flow[...,0], flow[...,1])
    #设置为hsv的第0个通道
    hsv[...,0] = ang*180/np.pi/2
    #设置为hsv的第2个通道
    hsv[...,2] = cv.normalize(mag,None,0,255,cv.NORM_MINMAX)
    bgr = cv.cvtColor(hsv,cv.COLOR_HSV2BGR)
    cv.imshow('frame2',bgr)
    k = cv.waitKey(30) & 0xff
    if k == 27:
        break
    elif k == ord('s'):
        cv.imwrite('opticalfb.png',frame2)
        cv.imwrite('opticalhsv.png',bgr)
    prvs = next
cap.release()
cv.destroyAllWindows()